# 01 – Local CLI in Docker

This notebook demonstrates the core features of Dockyter using a local Docker
installation:

- `%%docker` – run a whole cell in a single container.
- `%docker` – configure Docker mode for `!` commands.
- `%docker_status` – inspect the current configuration.
- `%docker_off` / `%docker_on` – enable/disable `!` redirection.
- basic argument validation (forbidden flags like `--privileged`).

**Prerequisites**

- Docker CLI available (for example, `docker --version` works in a terminal).
- The `dockyter` extension installed in this environment (`pip install dockyter`).


In [1]:
# Optional: check that Docker is available
!docker --version

Docker version 29.0.1, build eedd969


In [2]:
# Load the Dockyter IPython extension
%load_ext dockyter

## 1. Basic `%%docker` usage (single container per cell)

`%%docker` runs the **entire cell** in a single container. All commands share
the same shell state (e.g. `cd` affects the following `ls`).

Here we use the `ubuntu:22.04` image as a simple example.

In [3]:
%%docker ubuntu:22.04
echo "Hello from inside ubuntu:22.04"
pwd
ls /

Hello from inside ubuntu:22.04
/
bin
boot
dev
etc
home
lib
lib32
lib64
libx32
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var



## 2. `%docker` + `!` redirection

`%docker` configures Dockyter so that each subsequent `!cmd` is executed inside
a container with the given image and arguments.

It also automatically turns on Docker mode for `!`.

In [ ]:
%docker ubuntu:22.04

%docker_status

Connected

Docker runtime: Docker is installed and the daemon is running.
Docker redirection for '!': on
Current docker args: ubuntu:22.04


In [ ]:
!echo "This was executed inside ubuntu:22.04 via Dockyter"

This was executed inside ubuntu:22.04 via Dockyter



## 3. `!` commands do NOT share shell state

Each `!cmd` is run in a **fresh container**, so shell state does not persist
between `!` calls.

This is different from a `%%docker` cell, where all lines share the same
container and shell.

Below, `cd /tmp` does not affect the next `!pwd`, because they are separate
container runs.


In [ ]:
!cd /tmp
!pwd


/



If you need multiple commands to share state (directories, environment
variables, shell functions, etc.), use a single `%%docker` cell instead.

## 4. `%docker_off` and `%docker_on`

- `%docker_off` restores the original `!` behaviour (no Docker redirection).
- `%docker_on` re-activates Docker mode for `!` using the last configured
  image/arguments.


In [14]:
%docker_off

!dir
%docker_status

 Le volume dans le lecteur C n'a pas de nom.
 Le num�ro de s�rie du volume est FA0E-79E3

 R�pertoire de c:\Users\Simon\Documents\Projects\ProjetSemestre\hes-so-dockyter

02.12.2025  14:40    <DIR>          .
24.11.2025  15:19    <DIR>          ..
02.12.2025  14:11                19 .gitignore
26.11.2025  17:16    <DIR>          .venv
02.12.2025  16:55             9�034 01_local_cli.ipynb
02.12.2025  11:29             4�036 02_ml_tool_in_docker.ipynb
02.12.2025  15:19             4�841 dockyter.py
01.12.2025  16:28    <DIR>          exemples
24.11.2025  15:19             7�169 LICENSE
02.12.2025  15:23             5�022 README.md
02.12.2025  16:52    <DIR>          __pycache__
               6 fichier(s)           30�121 octets
               5 R�p(s)  122�608�889�856 octets libres
Docker runtime: Docker is installed and the daemon is running.
Docker redirection for '!': off
Current docker args: ubuntu:22.04


In [ ]:
%docker_on

!dir
%docker_status

bin   dev  home  lib32	libx32	mnt  proc  run	 srv  tmp  var
boot  etc  lib	 lib64	media	opt  root  sbin  sys  usr

Docker runtime: Docker is installed and the daemon is running.
Docker redirection for '!': on
Current docker args: ubuntu:22.04


## 5. Forbidden flags (basic argument validation)

Dockyter is **not** a security boundary, but it can still refuse obviously
dangerous Docker flags (for example `--privileged` or `--network=host`) to
avoid accidental misuse in shared environments.

If your Dockyter version implements a small denylist, trying to use such flags
should result in a clear error message instead of launching the container.


In [9]:
%docker --privileged ubuntu:22.04

Forbidden Docker flag detected: --privileged
Please remove this flag and try again.


In [10]:
%docker --network=host ubuntu:22.04

Forbidden Docker flag detected: --network=host
Please remove this flag and try again.
